In [6]:
import pandas as pd
import numpy as np
import math
from helpers import process_data

# Import cleaned up data

In [7]:
df = process_data('dataset_output/raw_training_data_cleaned_up.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100702 entries, 0 to 100701
Data columns (total 45 columns):
race                        100702 non-null int64
gender                      100702 non-null int64
age                         100702 non-null int64
admission_type_id           100702 non-null int64
discharge_disposition_id    100702 non-null int64
admission_source_id         100702 non-null int64
time_in_hospital            100702 non-null int64
num_lab_procedures          100702 non-null int64
num_procedures              100702 non-null int64
num_medications             100702 non-null int64
number_outpatient           100702 non-null int64
number_emergency            100702 non-null int64
number_inpatient            100702 non-null int64
diag_1                      100702 non-null int64
diag_2                      100702 non-null int64
diag_3                      100702 non-null int64
number_diagnoses            100702 non-null int64
max_glu_serum               100702 non-

#### All columns are using encoded int's. Great! 

# Gradient Tree Boosting

### 1) Construct training and testing 
TODO: validation sets

In [9]:
from sklearn.model_selection import train_test_split

# Separate input and output features 
# Original df is not mutated
X = df.drop(['discharge_disposition_id','admission_type_id'], axis=1)

# columns of the values we want to predict which we use to train our model
y_disposition = df['discharge_disposition_id']
y_admission = df['admission_type_id']

# For disposition statuses
X_disposition_train, X_disposition_test, y_disposition_train, y_disposition_test = train_test_split(X, y_disposition, test_size=0.3)

# For admission statuses
X_admission_train, X_admission_test, y_admission_train, y_admission_test = train_test_split(X, y_admission, test_size=0.3)

### 2) Train model

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

admission_classifier = GradientBoostingClassifier(n_estimators=100, verbose=1)
disposition_classifier = GradientBoostingClassifier(n_estimators=100, verbose=1)

admission_classifier.get_params()
disposition_classifier.get_params()

admission_classifier.fit(X_admission_train, y_admission_train)
disposition_classifier.fit(X_disposition_train, y_disposition_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

### 3) Predict!

In [11]:
# predict classes for data in test_set
predict_admission = admission_classifier.predict(X_admission_test) 
predict_disposition = disposition_classifier.predict(X_disposition_test)

### 4) Evaluate our models

In [15]:
from sklearn.metrics import classification_report
print(classification_report(predict_admission, y_admission_test))

             precision    recall  f1-score   support

          1       0.95      0.86      0.90     17563
          2       0.37      0.55      0.45      3727
          3       0.75      0.66      0.71      6415
          4       0.00      0.00      0.00         1
          5       0.66      0.68      0.67      1365
          6       0.71      0.97      0.82      1135
          7       0.00      0.00      0.00         0
          8       0.05      0.80      0.09         5

avg / total       0.81      0.78      0.79     30211



/Users/Domingo/deep_health/training_model/env/lib/python3.4/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
print(classification_report(predict_disposition, y_disposition_test))

             precision    recall  f1-score   support

          1       0.94      0.67      0.78     25075
          2       0.00      0.00      0.00         8
          3       0.44      0.46      0.45      3939
          4       0.00      0.07      0.01        15
          5       0.01      0.27      0.02        11
          6       0.08      0.39      0.14       828
          7       0.00      0.00      0.00         3
          8       0.00      0.00      0.00         0
          9       0.00      0.00      0.00         0
         11       0.06      0.40      0.11        77
         13       0.00      0.00      0.00         7
         14       0.02      0.40      0.03         5
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         1
         17       0.00      0.00      0.00         0
         18       0.06      0.37      0.11       182
         19       0.00      0.00      0.00         1
         22       0.01      0.30      0.03   

/Users/Domingo/deep_health/training_model/env/lib/python3.4/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Hmmm it seems that random forest does a better jobs in training multiple classes [in this case](http://scikit-learn.org/stable/modules/ensemble.html#classification)